In [1]:
import datetime
import math
import numpy as np
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F

In [2]:
class GNN(Module):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step #1
        self.hidden_size = hidden_size #100維度
        self.input_size = hidden_size * 2 #200
        self.gate_size = 3 * hidden_size #300
        # Parameter(output dim, input dim)
        self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size)) #weight:300,200
        self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size)) #weight:300,100
        self.b_ih = Parameter(torch.Tensor(self.gate_size)) #bias:300
        self.b_hh = Parameter(torch.Tensor(self.gate_size)) #bias:300
        self.b_iah = Parameter(torch.Tensor(self.hidden_size)) #bias:100
        self.b_oah = Parameter(torch.Tensor(self.hidden_size)) #bias:100

        self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True) #(100,100)+100
        self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True) #(100,100)+100
        self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True) #(100,100)+100

    def GNNCell(self, A, hidden):
        #A.shape = batch_size * max_n_node * (2 max_n_node)
        #hidden.shape = batch_size * max_n_node * 100
        #A[:, :, :A.shape[1]]: batch_size * max_n_node * max_n_node
        #self.linear_edge_in(hidden): batch_size * max_n_node * 100
        #input_in.shape: batch_size * max_n_node * 100
        input_in = torch.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = torch.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]], self.linear_edge_out(hidden)) + self.b_oah
        #inputs.shape: batch_size * max_n_node * 200 
        inputs = torch.cat([input_in, input_out], 2)
        #self.w_ih.shape: 200 * 300
        #gi.shape: batch_size * max_n_node * 300
        gi = F.linear(inputs, self.w_ih, self.b_ih)
        #self.w_hh.shape: 200 * 300
        #gh.shape: batch_size * max_n_node * 300
        gh = F.linear(hidden, self.w_hh, self.b_hh)
        #dim(i_r): batch_size * max_n_node * 100
        i_r, i_i, i_n = gi.chunk(3, 2) #torch.chunk(): 沿著dim=2，平切分3塊
        h_r, h_i, h_n = gh.chunk(3, 2)
        resetgate = torch.sigmoid(i_r + h_r)
        inputgate = torch.sigmoid(i_i + h_i)
        newgate = torch.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def forward(self, A, hidden):
        for i in range(self.step): #step:1
            hidden = self.GNNCell(A, hidden)
        return hidden

class SessionGraph(Module):
    #opt: parser.parse_args('')
    def __init__(self, opt, n_node):
        super(SessionGraph, self).__init__()
        self.hidden_size = opt.hiddenSize #100
        self.n_node = n_node #310
        self.batch_size = opt.batchSize #100
        self.nonhybrid = opt.nonhybrid #False
        self.embedding = nn.Embedding(self.n_node, self.hidden_size) #有310個node，轉成100個維度表示
        self.gnn = GNN(self.hidden_size, step=opt.step) #hidden_size:100 , step:1
        self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True) #(100,100)+100
        self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True) #(100,100)+100
        self.linear_three = nn.Linear(self.hidden_size, 1, bias=False) #(100,1)
        self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True) #(200,100)+100
        self.loss_function = nn.CrossEntropyLoss() #loss_function():CrossEntropyLoss()
        #optimizer: 1.self.parameters() 2.lr=0.001 3.weight_decay=1e-5
        self.optimizer = torch.optim.Adam(self.parameters(), lr=opt.lr, weight_decay=opt.l2) 
        #scheduler: 調整learning rate。 1.self.optimizer 2.step_size=3(每三個epoch下降lr) 3.gamma=0.1(每次下降為原本的0.1倍)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=opt.lr_dc_step, gamma=opt.lr_dc)
        #先初始化parameters
        self.reset_parameters() #函數先建立在下方

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def compute_scores(self, hidden, mask):
        #hidden:最後輸出的結果，對於每一個item_chain得到一個100維度的結果
        #mask.shape[0] = batch_size
        #ht: input item的最後一個hidden(100維)結果（不包含補0）
        #ht.shape: batch_size * 100
        ht = hidden[torch.arange(mask.shape[0]).long(), torch.sum(mask, 1) - 1] 
        #ht -->fc--> q1
        #q1: 每個item chain的最後一個item都打壓成(1 * 100)  # batch_size * 1 * 100
        q1 = self.linear_one(ht).view(ht.shape[0], 1, ht.shape[1])
        #hidden.shape: batch_size * max_n_node * 100
        #hidden --> fc --> q2
        #q2: item chain的全部item都在一次fully connected轉換 # batch_size * max_n_node * 100
        q2 = self.linear_two(hidden)  
        #q1+q2: batch_size * max_n_node * 100 (q2中每個item都加上q1)
        #alpha: batch_size * max_n_node * 1
        alpha = self.linear_three(torch.sigmoid(q1 + q2))
        #alpha * hidden * mask.view(mask.shape[0], -1, 1).float()
        #(batch_size*max_n_node*1) * (batch_size*max_n_node * 100) * (batch_size*max_n_node*1)
        #a: mask=1的items 100維相加
        #a.shape: batch_size * 100維
        a = torch.sum(alpha * hidden * mask.view(mask.shape[0], -1, 1).float(), 1)
        if not self.nonhybrid:
            #如果nonhybrid = Fasle:
            #將a改成 concatenate(a, ht): shape = batch_size * 200維
            a = self.linear_transform(torch.cat([a, ht], 1))
        #b: 不包含0，為1~309對應的100維字典
        b = self.embedding.weight[1:]
        #a維度: batch_size * 200
        #b.transpose(1, 0)維度: 100 * 309
        #scores維度: batch_size * 309
        scores = torch.matmul(a, b.transpose(1, 0))
        return scores

    def forward(self, inputs, A):
        #input是一個items:單一items為[0, 17, 18, 0, 0, ...]
        #self.embedding: 查找0~309數字，所對應在100維空間的字典
        #inputs.shape = batch_size * max_n_node 
        hidden = self.embedding(inputs) #hidden.shape = batch_size * max_n_node * 100
        hidden = self.gnn(A, hidden) #執行GNN.forward(A, hidden)
        return hidden


def trans_to_cuda(variable):
    if torch.cuda.is_available():
        return variable.cuda()
    else:
        return variable


def trans_to_cpu(variable):
    if torch.cuda.is_available():
        return variable.cpu()
    else:
        return variable


def forward(model, i, data):
    #i: 一個batch的index
    #data: Data類別的train_data或test_data
    
    #單一alias_inputs: [1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    #單一A:concate(incoming, outgoing)
    #單一items: [0, 17, 18, 0, 0, ...]
    #單一mask: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    #單一targets: 17
    alias_inputs, A, items, mask, targets = data.get_slice(i)
    alias_inputs = trans_to_cuda(torch.Tensor(alias_inputs).long())
    items = trans_to_cuda(torch.Tensor(items).long())
    A = trans_to_cuda(torch.Tensor(A).float())
    mask = trans_to_cuda(torch.Tensor(mask).long())
    #model(items, A): 執行SessionGraph.forward()
    hidden = model(items, A)
    #執行完gnn的hidden: shape=batch_size * max_n_node * 100
    get = lambda i: hidden[i][alias_inputs[i]]
    seq_hidden = torch.stack([get(i) for i in torch.arange(len(alias_inputs)).long()])
    #seq_hidden把下面的alias的每個item表達成hidden 100維的形式
    #alias_inputs: [1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    #targets維度: batch_size * 1
    #scores維度: batch_size * 309
    return targets, model.compute_scores(seq_hidden, mask)


def train_test(model, train_data, test_data):
    #model.scheduler.step(): 依照model設定好的scheduler更新learning rate
    model.scheduler.step()
    print('start training: ', datetime.datetime.now())
    #model.train(): 開啟訓練模式
    model.train()
    #total_loss: 之後每個batch的loss會加入
    total_loss = 0.0
    #slices: 為呼叫inputs, mask, targets的batch index
    slices = train_data.generate_batch(model.batch_size) #opt設定的batch_size為100
    for i, j in zip(slices, np.arange(len(slices))):
        #i為一個batch的index
        #j為第幾個batch，從0開始
        model.optimizer.zero_grad()
        #targets維度: batch_size * 1
        #scores維度: batch_size * 309
        targets, scores = forward(model, i, train_data)
        targets = trans_to_cuda(torch.Tensor(targets).long())
        #scores:309個位置
        #targets - 1: 目標(正確答案)的位置
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        total_loss += loss
        if j % int(len(slices) / 5 + 1) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), loss.item()))
    print('Total Loss:\t%.3f' % total_loss)

    print('start predicting: ', datetime.datetime.now())
    model.eval()
    hit, mrr = [], []
    slices = test_data.generate_batch(model.batch_size)
    for i in slices:
        targets, scores = forward(model, i, test_data)
        #sub_scores: 分數最高的前20名之index
        sub_scores = scores.topk(20)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
    
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            #score:分數前20的index shape=20
            #targets:正確答案 shape=1
            #score代表一次推薦20個產品
            #hit代表，推薦的20個產品中，有沒有打中正確答案
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                #如果推薦項目沒中，mrr append 0
                mrr.append(0)
            else:
                #如果推薦項目中在第0位: append 1
                #如果推薦項目中在第1位: append 1/2
                #如果推薦項目中在第2位: append 1/3
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    #hit: 推薦有中的比率 0~100%
    #mrr: 推薦有中的準確度 0~100%
    return hit, mrr


In [3]:
def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max

def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0] 
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs) #inputs: 補零後的自變相items_index chain(x)
        self.mask = np.asarray(mask) #mask: 自變相items_index chain中，有items_index為1；補零值為0
        self.len_max = len_max #len_max: 最長的自變相items_index chain
        self.targets = np.asarray(data[1]) #targets: 預測的正解
        self.length = len(inputs) #length: 樣本數
        self.shuffle = shuffle #shuffle: True/False
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg) #新的排序
            self.inputs = self.inputs[shuffled_arg] 
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size) # n_batch: 共有__個batch
        if self.length % batch_size != 0:
            n_batch += 1
        #np.split(list, int): 把list平均切成int份
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        #slices: 為呼叫inputs, mask, targets的batch index
        return slices

    def get_slice(self, i):
        #i: 為單一batch呼叫inputs, mask, targets的index
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            #u_input範例: array([17, 17, 18, 18, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
            #n_node: 在一個session中，unique(item_index)數目
            n_node.append(len(np.unique(u_input))) #範例=3
        #max_n_node: 在某一個batch中，最大的n_node
        max_n_node = np.max(n_node) 
        for u_input in inputs:
            #node: 在一個session中，unique(item_index)
            node = np.unique(u_input) #範例: array([0, 17, 18])
            #items: 每個session中，都去除了重複item後的item chain，剩下補0，讓一個batch中，chain的長度都一樣
            items.append(node.tolist() + (max_n_node - len(node)) * [0]) #範例[0, 17, 18, 0, 0, ......]
            #u_A: 某個session的A矩陣
            u_A = np.zeros((max_n_node, max_n_node))
            for i in np.arange(len(u_input) - 1): 
                #假設: u_input=array([17, 17, 18, 18, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
                #假設: len(u_input)=16
                #假設: i範圍=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])
                if u_input[i + 1] == 0:
                    break
                #假設node=array([0, 17, 18])
                #u_A為有向的A矩陣
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            #row正規化
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            #製作connection matrix A:左半邊為incoming edges；右半邊為outgoing edges
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            #假設: u_input=array([17, 17, 18, 18, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
            #假設: alias_inputs=[1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets
                #alias_inputs: [1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                #A:
                #items: [0, 17, 18, 0, 0, ...]
                #mask: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                #targets: 17

In [4]:
import argparse
import pickle
import time


In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='sample', help='dataset name: diginetica/yoochoose1_4/yoochoose1_64/sample')
parser.add_argument('--batchSize', type=int, default=100, help='input batch size')
parser.add_argument('--hiddenSize', type=int, default=100, help='hidden state size')
parser.add_argument('--epoch', type=int, default=30, help='the number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')  # [0.001, 0.0005, 0.0001]
parser.add_argument('--lr_dc', type=float, default=0.1, help='learning rate decay rate')
parser.add_argument('--lr_dc_step', type=int, default=3, help='the number of steps after which the learning rate decay')
parser.add_argument('--l2', type=float, default=1e-5, help='l2 penalty')  # [0.001, 0.0005, 0.0001, 0.00005, 0.00001]
parser.add_argument('--step', type=int, default=1, help='gnn propogation steps')
parser.add_argument('--patience', type=int, default=10, help='the number of epoch to wait before early stop ')
parser.add_argument('--nonhybrid', action='store_true', help='only use the global preference to predict')
parser.add_argument('--validation', action='store_true', help='validation')
parser.add_argument('--valid_portion', type=float, default=0.1, help='split the portion of training set as validation set')
opt = parser.parse_args('')
print(opt)


Namespace(batchSize=100, dataset='sample', epoch=30, hiddenSize=100, l2=1e-05, lr=0.001, lr_dc=0.1, lr_dc_step=3, nonhybrid=False, patience=10, step=1, valid_portion=0.1, validation=False)


In [6]:
#load training set
train_data = pickle.load(open('pickle_dataset/' + opt.dataset + '/train.txt', 'rb'))
#load training set or validation set
if opt.validation:
    train_data, valid_data = split_validation(train_data, opt.valid_portion)
    test_data = valid_data
else:
    test_data = pickle.load(open('pickle_dataset/' + opt.dataset + '/test.txt', 'rb'))
#轉成Data
train_data = Data(train_data, shuffle=True)
test_data = Data(test_data, shuffle=False)    


In [7]:
n_node = 310 
#SessionGraph為訓練模型
model = trans_to_cuda(SessionGraph(opt, n_node))
model

SessionGraph(
  (embedding): Embedding(310, 100)
  (gnn): GNN(
    (linear_edge_in): Linear(in_features=100, out_features=100, bias=True)
    (linear_edge_out): Linear(in_features=100, out_features=100, bias=True)
    (linear_edge_f): Linear(in_features=100, out_features=100, bias=True)
  )
  (linear_one): Linear(in_features=100, out_features=100, bias=True)
  (linear_two): Linear(in_features=100, out_features=100, bias=True)
  (linear_three): Linear(in_features=100, out_features=1, bias=False)
  (linear_transform): Linear(in_features=200, out_features=100, bias=True)
  (loss_function): CrossEntropyLoss()
)

In [8]:
start = time.time()
best_result = [0, 0]
best_epoch = [0, 0]
bad_counter = 0
#opt.epoch:30
for epoch in range(opt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)
    hit, mrr = train_test(model, train_data, test_data)
    print('hit: %.4f%%'%hit)
    print('mrr: %.4f%%'%mrr)
    flag = 0
    if hit >= best_result[0]:
        best_result[0] = hit
        best_epoch[0] = epoch
        flag = 1
    if mrr >= best_result[1]:
        best_result[1] = mrr
        best_epoch[1] = epoch
        flag = 1
    print('Best Result:')
    print('\tRecall@20:\t%.4f%%\tMMR@20:\t%.4f%%\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))
    bad_counter += 1 - flag
    if bad_counter >= opt.patience:
        break
print('-------------------------------------------------------')
end = time.time()
print("Run time: %f s" % (end - start))

-------------------------------------------------------
epoch:  0
start training:  2021-05-27 11:56:46.511235


C:\Users\chienhua\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[0/10] Loss: 5.7345
[3/10] Loss: 5.7275
[6/10] Loss: 5.7105
[9/10] Loss: 5.6589
Total Loss:	57.052
start predicting:  2021-05-27 11:56:47.663709
hit: 3.4091%
mrr: 0.9470%
Best Result:
	Recall@20:	3.4091%	MMR@20:	0.9470%	Epoch:	0,	0
-------------------------------------------------------
epoch:  1
start training:  2021-05-27 11:56:47.712710
[0/10] Loss: 5.5445
[3/10] Loss: 5.4488
[6/10] Loss: 5.3678
[9/10] Loss: 5.4692
Total Loss:	54.815
start predicting:  2021-05-27 11:56:48.661710
hit: 6.8182%
mrr: 0.6140%
Best Result:
	Recall@20:	6.8182%	MMR@20:	0.9470%	Epoch:	1,	0
-------------------------------------------------------
epoch:  2
start training:  2021-05-27 11:56:48.717710
[0/10] Loss: 5.2942
[3/10] Loss: 5.1593
[6/10] Loss: 5.1434
[9/10] Loss: 5.1491
Total Loss:	52.370
start predicting:  2021-05-27 11:56:49.673710
hit: 6.8182%
mrr: 0.6853%
Best Result:
	Recall@20:	6.8182%	MMR@20:	0.9470%	Epoch:	2,	0
-------------------------------------------------------
epoch:  3
start training:  2